In [1]:
import os
os.chdir("../function")

In [2]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:           47Gi        13Gi        22Gi       3.0Mi        11Gi        33Gi
스왑:        16Gi       4.0Gi        12Gi


In [6]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import time
import load_data
import Vital_DF
import multiprocessing
from mapping import generate_mapping_function
from mapping_dict import get_breed_dict

In [44]:
dataconn = load_data.LoadData()

ShowDB_Df = dataconn.select_data(
    "show databases"
    )
Schema_all = ShowDB_Df.iloc[0:-4,:].values.tolist()
Schema_all = sum(Schema_all, [])

# disconnect from server
dataconn.close()

In [45]:
data = None
start = time.time()
with multiprocessing.Pool(48)  as pool:
    data = list(pool.map(Vital_DF.makeDf, Schema_all))
end = time.time()
print(end - start)

989.vital doesn't exist

5120.603721857071


In [46]:
data = pd.concat(data)
data

,SNo,Name,Name2,Name3,Sex,Day,VT_BW
0,333,둥이,Maltese,몰티즈,3,565,3.90
1,333,둥이,Maltese,몰티즈,3,565,4.10
2,457,김밍키,Maltese,몰티즈,3,4451,3.66
3,117,뽀글이,Maltese,몰티즈,2,1557,7.00
4,469,뭉치,Shih Tzu,시추,5,4748,5.30
...,...,...,...,...,...,...,...
4988,4515,홍시,Bichon Frise,비숑 프리제,4,1096,4.82
4989,4516,봉달,Poodle,푸들,2,105,1.18
4990,4529,곱슬이,Poodle,푸들,0,731,2.38
4991,4536,손다숑,Bichon Frise,비숑 프리제,2,88,0.68


In [47]:
data.isnull().sum()

SNo          0
Name         0
Name2        0
Name3        0
Sex          0
Day          0
VT_BW    75434
dtype: int64

In [48]:
data.dropna(inplace=True)

In [49]:
data = data.reset_index(drop=True)

In [66]:
data

,SNo,Name,Name2,Name3,Sex,Day,VT_BW,Month
0,333,Maltese,Maltese,몰티즈,3,565,3.90,18
1,333,Maltese,Maltese,몰티즈,3,565,4.10,18
2,457,Maltese,Maltese,몰티즈,3,4451,3.66,148
3,117,Maltese,Maltese,몰티즈,2,1557,7.00,51
5,182,Maltese,Maltese,몰티즈,3,3351,4.17,111
...,...,...,...,...,...,...,...,...
12003132,4508,Maltese,Maltese,말티즈,3,2198,2.86,73
12003133,4510,Golden_Retriever,Golden Retriever,골든 레트리버,3,2192,33.00,73
12003135,4515,Bichon_Frise,Bichon Frise,비숑 프리제,4,1096,4.82,36
12003138,4536,Bichon_Frise,Bichon Frise,비숑 프리제,2,88,0.68,2


In [67]:
mapping_dict = get_breed_dict()

tqdm.pandas()
data["Name"] = data.progress_apply(lambda row : generate_mapping_function(row,mapping_dict),axis=1)

100%|██████████| 7693106/7693106 [07:33<00:00, 16954.90it/s]


In [68]:
data

,SNo,Name,Name2,Name3,Sex,Day,VT_BW,Month
0,333,Maltese,Maltese,몰티즈,3,565,3.90,18
1,333,Maltese,Maltese,몰티즈,3,565,4.10,18
2,457,Maltese,Maltese,몰티즈,3,4451,3.66,148
3,117,Maltese,Maltese,몰티즈,2,1557,7.00,51
5,182,Maltese,Maltese,몰티즈,3,3351,4.17,111
...,...,...,...,...,...,...,...,...
12003132,4508,Maltese,Maltese,말티즈,3,2198,2.86,73
12003133,4510,Golden_Retriever,Golden Retriever,골든 레트리버,3,2192,33.00,73
12003135,4515,Bichon_Frise,Bichon Frise,비숑 프리제,4,1096,4.82,36
12003138,4536,Bichon_Frise,Bichon Frise,비숑 프리제,2,88,0.68,2


In [69]:
data = data[(data["Sex"].isin([1,2,3,4])) & (data["Day"] > 0)]
data = data[data["Name"].isnull() == False]
data = data[data["Name"] != "Unknown"]
data

,SNo,Name,Name2,Name3,Sex,Day,VT_BW,Month
0,333,Maltese,Maltese,몰티즈,3,565,3.90,18
1,333,Maltese,Maltese,몰티즈,3,565,4.10,18
2,457,Maltese,Maltese,몰티즈,3,4451,3.66,148
3,117,Maltese,Maltese,몰티즈,2,1557,7.00,51
5,182,Maltese,Maltese,몰티즈,3,3351,4.17,111
...,...,...,...,...,...,...,...,...
12003132,4508,Maltese,Maltese,말티즈,3,2198,2.86,73
12003133,4510,Golden_Retriever,Golden Retriever,골든 레트리버,3,2192,33.00,73
12003135,4515,Bichon_Frise,Bichon Frise,비숑 프리제,4,1096,4.82,36
12003138,4536,Bichon_Frise,Bichon Frise,비숑 프리제,2,88,0.68,2


In [72]:
data.loc[data["VT_BW"] > 100,"VT_BW"] *= 0.001
data = data[(data["Sex"].isin([1,2,3,4])) & (data["Day"] > 0)]

In [74]:
data["Month"] = data["Day"]//30

In [75]:
data

,SNo,Name,Name2,Name3,Sex,Day,VT_BW,Month
0,333,Maltese,Maltese,몰티즈,3,565,3.90,18
1,333,Maltese,Maltese,몰티즈,3,565,4.10,18
2,457,Maltese,Maltese,몰티즈,3,4451,3.66,148
3,117,Maltese,Maltese,몰티즈,2,1557,7.00,51
5,182,Maltese,Maltese,몰티즈,3,3351,4.17,111
...,...,...,...,...,...,...,...,...
12003132,4508,Maltese,Maltese,말티즈,3,2198,2.86,73
12003133,4510,Golden_Retriever,Golden Retriever,골든 레트리버,3,2192,33.00,73
12003135,4515,Bichon_Frise,Bichon Frise,비숑 프리제,4,1096,4.82,36
12003138,4536,Bichon_Frise,Bichon Frise,비숑 프리제,2,88,0.68,2


In [76]:
def remove_outlier(df=None, dog=None, weight=1.5):
    # target 값과 상관관계가 높은 열을 우선적으로 진행
    quantile_25 = np.percentile(df[df["Name"] == dog]["VT_BW"].values, 25)
    quantile_75 = np.percentile(df[df["Name"] == dog]["VT_BW"].values, 75)

    IQR = quantile_75 - quantile_25
    IQR_weight = IQR*weight

    lowest = quantile_25 - IQR_weight
    highest = quantile_75 + IQR_weight

    r_outlier = df[(df["Name"] == dog) & (df["VT_BW"] > lowest) & (df["VT_BW"] < highest)]
    return r_outlier

In [77]:
df = pd.DataFrame()
for dog in mapping_dict.keys():
    try:
        temp = remove_outlier(data,dog)
        df = pd.concat([df,temp])
    except:
        continue
df

,SNo,Name,Name2,Name3,Sex,Day,VT_BW,Month
1620,291,Bichon_Frise,Bichon Frise,비숑 프리제,3,96,1.19,3
1621,291,Bichon_Frise,Bichon Frise,비숑 프리제,3,110,1.74,3
1622,291,Bichon_Frise,Bichon Frise,비숑 프리제,3,447,4.70,14
1623,291,Bichon_Frise,Bichon Frise,비숑 프리제,3,593,5.00,19
1624,291,Bichon_Frise,Bichon Frise,비숑 프리제,3,875,4.90,29
...,...,...,...,...,...,...,...,...
12002934,3850,Yorkshire Terrier,Yorkshire Terrier,요크셔 테리어,4,3511,2.20,117
12003029,4130,Yorkshire Terrier,Yorkshire Terrier,요크셔 테리어,4,731,2.80,24
12003054,4177,Yorkshire Terrier,Yorkshire Terrier,요크셔 테리어,3,3288,4.36,109
12003055,4177,Yorkshire Terrier,Yorkshire Terrier,요크셔 테리어,3,3331,4.50,111


In [78]:
df["_Year"] = df["Day"] //365
df.rename(columns = {"Month":"_Month"},inplace=True)

In [79]:
df = df[df["_Year"] <= 30]

In [83]:
df

,SNo,Name,Name2,Name3,Sex,Day,VT_BW,_Month,_Year
1620,291,Bichon_Frise,Bichon Frise,비숑 프리제,3,96,1.19,3,0
1621,291,Bichon_Frise,Bichon Frise,비숑 프리제,3,110,1.74,3,0
1622,291,Bichon_Frise,Bichon Frise,비숑 프리제,3,447,4.70,14,1
1623,291,Bichon_Frise,Bichon Frise,비숑 프리제,3,593,5.00,19,1
1624,291,Bichon_Frise,Bichon Frise,비숑 프리제,3,875,4.90,29,2
...,...,...,...,...,...,...,...,...,...
12002934,3850,Yorkshire Terrier,Yorkshire Terrier,요크셔 테리어,4,3511,2.20,117,9
12003029,4130,Yorkshire Terrier,Yorkshire Terrier,요크셔 테리어,4,731,2.80,24,2
12003054,4177,Yorkshire Terrier,Yorkshire Terrier,요크셔 테리어,3,3288,4.36,109,9
12003055,4177,Yorkshire Terrier,Yorkshire Terrier,요크셔 테리어,3,3331,4.50,111,9


In [80]:
df.to_csv("../File/Vital_named_cleaned.csv",index=False)

In [81]:
df.groupby(["Name","_Month"], as_index=False)["VT_BW"].mean().head(60)

,Name,_Month,VT_BW
0,BULLDOG,0,0.882561
1,BULLDOG,1,1.936092
2,BULLDOG,2,2.410556
3,BULLDOG,3,3.809298
4,BULLDOG,4,5.411286
5,BULLDOG,5,6.902386
6,BULLDOG,6,8.081809
7,BULLDOG,7,9.014729
8,BULLDOG,8,9.572980
9,BULLDOG,9,10.093938
